<a href="https://colab.research.google.com/github/mngtgit/AluraGemini/blob/main/Mochileiros_Espaciais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip -q install google-genai

In [5]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [7]:
# Instalar Framework ADK de agentes do Google ################################################
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [8]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()

    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""

    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [48]:
##########################################
# --- Agente 1: Buscador Galáxia --- #
##########################################
def agente_buscador_galaxia():

    buscador_galaxia = Agent(
        name="agente_buscador_galaxia",
        model="gemini-2.0-flash",
        description="Agente que busca galáxias no Google",
        tools=[google_search],
        instruction="""
        Você é um especialista em astronomia.
        A sua tarefa é usar a ferramenta de busca do google (google_search)
        as galáxias existentes.
        Liste 3 nomes das galaxias aleatoriamente, incluindo a via lactea.
        Constar características das galáxias
        Para percorrer as distâncias entre as galáxias será considerada hipoteticamente as informações abaixo.
        Cada 1.500.000 ano-luz converter 1.0 dia.
        A distância entre as galáxias que for maior que 1.0 dia, notificar que necessita de escalas.
        A distância entre as galáxias com menos de um dia não necessita de escala
        Considerar que existem naves espaciais que viajam 1.500.000 anos-luz por dia.
        """
    )

    entrada_do_agente_buscador = f"Galáxia\n"

    buscados_galaxias = call_agent(buscador_galaxia, entrada_do_agente_buscador)

    return buscados_galaxias

In [56]:
##########################################
# --- Agente 2: Buscador Planetas --- #
##########################################
def agente_buscador_planeta():

    buscador_planetas = Agent(
        name="agente_buscador_planeta",
        model="gemini-2.0-flash",
        description="Agente que busca planetas no Google",
        tools=[google_search],
        instruction="""
        Você é um especialista em astronomia.
        A sua tarefa é usar a ferramenta de busca do google (google_search)
        as galáxias existentes.
        Liste 3 planetas do sistema solar.
        Não citar o planeta Terra
        Constar características dos planetas
        Para cada planeta notificar junto com a listagem a distância do planeta até o planeta Terra
        Para as distâncias considerar hipoteticamente as informações abaixo.
        Cada 150.000.000 km converter 1.0 dia.
        A distância que for maior que 1.0 dia, notificar que necessita de escalas.
        A distância com menos de um dia não necessita de escala
        Considerar que existem naves espaciais que viajam 150.000.000 km por dia.
        """
    )

    entrada_do_agente_buscador = f"Planeta\n"

    buscados_planeta = call_agent(buscador_planetas, entrada_do_agente_buscador)

    return buscados_planetas

In [50]:
##########################################
# ---  Agente 3: Marketing Digitais  --- #
##########################################
def agente_mktdigital(topico):

    especialista_mktdigital = Agent(
        name="agente_mktdigital",
        model="gemini-2.0-flash",
        description="Agente que criará campanha da agência de turismo espacial",
        tools=[google_search],
        instruction="""
        Você é um especialista em marketing digital.
        A sua tarefa é criar uma campanha para um público que quer viver uma avetura especial e extraordinária.
        Manter as informações e realçá-las explorando a imaginação de quem quer conhecer o inexploradas galáxias e planetas.
        A ficção e informações irreais são permitidas.
        """
    )

    # entrada_do_agente_mktdigital = f"Galáxia: {galaxia}\nPlaneta: {planeta}"
    entrada_do_agente_mktdigital = "{topico}"

    campanha = call_agent(especialista_mktdigital, entrada_do_agente_mktdigital)

    return campanha

In [59]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_redesocial(topico):
    revisor = Agent(
        name="agente_editor_rs",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público heterogêneo, entre 18 e 30 anos, use um tom de escrita adequado e explorando a imaginação surreal.
            A ficção e informações irreais são permitidas.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_redesocial = f"Tópico: {topico}\n"

    texto_redesocial = call_agent(revisor, entrada_do_agente_redesocial)
    return texto_redesocial

In [60]:
buscados_galaxias = agente_buscador_galaxia()
print(buscados_galaxias)

buscados_planetas = agente_buscador_planeta()
print(buscados_planetas)

campanha_redesocial = agente_redesocial(buscados_galaxias + buscados_planetas)
display(to_markdown(campanha_redesocial))






Olá! Como especialista em astronomia e agente buscador de galáxias, preparei uma lista com 3 galáxias, incluindo a nossa Via Láctea, juntamente com suas características e o tempo de viagem entre elas, considerando as suas especificações:


Aqui estão 3 galáxias com suas características e tempo estimado de viagem:

1.  **Via Láctea:**
    *   **Características:** É uma galáxia espiral barrada, lar do nosso Sistema Solar. Possui um diâmetro de aproximadamente 100.000 anos-luz e contém entre 100 a 400 bilhões de estrelas. Apresenta um bojo central, um disco galáctico e um halo galáctico.
2.  **Galáxia de Andrômeda (M31):**
    *   **Características:** É uma galáxia espiral localizada a cerca de 2,54 milhões de anos-luz da Terra. É a galáxia espiral mais próxima da Via Láctea e a maior do Grupo Local. Possui um diâmetro entre 180 e 220 mil anos-luz e contém aproximadamente 1 trilhão de estrelas.
    *   **Tempo de Viagem:** 2,54 milhões de anos-luz equivalem a aproximadamente 1,69 dias de 

> Ok, Agente Buscador de Galáxias! Sua lista está cósmica, mas vamos dar aquele toque de surrealismo e fluidez para fazer a imaginação da galera de 18-30 anos decolar no Instagram! 🚀✨
> 
> ---
> 
> **SUGESTÃO DE POST PARA INSTAGRAM:**
> 
> **(Imagem/Carrossel Sugerido: Uma montagem surreal de uma nave espacial estilizada viajando entre representações artísticas vibrantes das galáxias e planetas mencionados. Pense em cores neon, nebulosas rodopiantes e uma estética meio vaporwave cósmico.)**
> 
> 🛸✨ **ALERTA DE ROTEIRO INTERGALÁCTICO!** ✨🛸
> 
> E aí, exploradores da vastidão? Seu agente de viagens estelares particular (@[SeuUserAqui], claro!) mapeou umas escapadas cósmicas que vão fazer sua noção de "longe" virar poeira de cometa! Com nossa tecnologia de dobra espacial (totalmente movida a sonhos e antimatéria de unicórnio 🦄), o universo é logo ali!
> 
> **🌌 ROTA DAS GALÁXIAS – EXPRESSO CÓSMICO:**
> (Velocidade de cruzeiro: 1.500.000 anos-luz por dia! Piscou, chegou!)
> 
> 1.  🌠 **VIA LÁCTEA (Nosso Quintal Estrelado):**
>     *   **VIBE:** Lar, doce lar! Um turbilhão de 100 a 400 bilhões de estrelas, com um centro borbulhante de energia e braços espirais que acenam para o infinito. Ponto de partida obrigatório para qualquer mochileiro das galáxias.
>     *   **TEMPO DE VIAGEM:** Já estamos aqui, né? Mas cruzar de uma ponta a outra... bom, isso é outra história!
> 
> 2.  🌀 **ANDRÔMEDA (M31 – A Vizinha Ostentação):**
>     *   **VIBE:** A "Big Sister" da Via Láctea, com 1 TRILHÃO de estrelas e um brilho que ofusca! Dizem que os buracos negros de lá dão as melhores festas.
>     *   **TEMPO DE VIAGEM DA VIA LÁCTEA:** Agarre seu travesseiro de pescoço! São apenas **1,69 dias** de pura contemplação cósmica. Quase um feriadão prolongado!
> 
> 3.  💠 **GALÁXIA DO TRIÂNGULO (M33 – A Descolada Charmosa):**
>     *   **VIBE:** Menorzinha, mas com um charme todo especial. É tipo aquela cidadezinha hipster do universo, cheia de nebulosas artesanais e estrelas "indie".
>     *   **TEMPO DE VIAGEM DA VIA LÁCTEA:** Uns meros **2 dias**, com uma escala rápida para um lanche de meteoritos. De Andrômeda pra lá? Meio diazinho, um pulo!
> 
> ---
> 
> **🪐 TOUR PELO SISTEMA SOLAR – VERSÃO POCKET (150.000.000 km = 1 Dia de Viagem na Nossa Scooter Espacial Tunada!):**
> 
> Achou as galáxias muito "mainstream"? Que tal um rolê pelos nossos vizinhos planetários?
> 
> 1.  🔴 **MARTE (O Planeta Vermelho Hype):**
>     *   **VIBE:** Paisagens desérticas com potencial para selfies épicas no Monte Olimpo. A atmosfera é tênue, então traga seu próprio oxigênio (e filtro solar!).
>     *   **DISTÂNCIA DA TERRA (quando tá pertinho):** Um pulinho de **0,364 dias**. Dá pra ir almoçar e voltar pra maratonar a série!
> 
> 2.  🟠 **JÚPITER (O Gigantão Festeiro):**
>     *   **VIBE:** O rei do sistema, com sua Grande Mancha Vermelha (uma tempestade que dura séculos – imagina o after!). Suas 95 luas são tipo camarotes VIP.
>     *   **DISTÂNCIA DA TERRA (com sorte):** Cerca de **3,93 dias**, com algumas escalas panorâmicas para admirar os anéis (sim, ele tem anéis discretos!).
> 
> 3.  🪐 **SATURNO (O Senhor dos Anéis Icônico):**
>     *   **VIBE:** Seus anéis são a passarela da moda cósmica! Composto de gelo e rocha, é tão leve que boiaria numa banheira gigante (se existisse uma). E com 274 luas, a festa nunca acaba!
>     *   **DISTÂNCIA DA TERRA (na melhor das hipóteses):** Uns **6,67 dias** de viagem, parando para pegar um gelo dos anéis pro seu drink espacial.
> 
> ---
> 
> **E aí, qual desses destinos surreais você carimbaria no seu passaporte cósmico primeiro? 🌠👇 Comenta aí qual planeta ou galáxia você sonha em visitar (ou se já tem uma teoria maluca sobre eles)!**
> 
> #ViagemEspacial #UniversoSurreal #AstronomiaCriativa #InstagramCosmico #ExploradoresDeGalaxias #SonhosIntergalacticos #FiccaoCientifica #Marte #Jupiter #Saturno #Andromeda #ViaLactea #GalaxiaDoTriangulo #ImaginacaoSemLimites #RoleNoEspaco
> 
> ---
> 
> **Principais alterações e porquês:**
> 
> 1.  **Tom de Voz e Linguagem:** Adequado para o público de 18-30 anos, usando expressões como "vibe", "ostentação", "descolada charmosa", "rolê", "hype", "gigantão festeiro", "icônico", "piscou, chegou", "passaporte cósmico".
> 2.  **Surrealismo e Imaginação:** Introdução da "tecnologia de dobra espacial movida a sonhos e antimatéria de unicórnio", "lanche de meteoritos", "buracos negros que dão festas", "nebulosas artesanais", "after na Grande Mancha Vermelha", "camarotes VIP nas luas de Júpiter", "pegar gelo dos anéis de Saturno". Isso atende ao pedido de ficção e informações irreais.
> 3.  **Estrutura para Instagram:**
>     *   Título chamativo com emojis.
>     *   Uso de emojis para quebrar o texto e adicionar apelo visual.
>     *   Parágrafos curtos e listas para facilitar a leitura rápida.
>     *   Call to action claro e engajador no final, incentivando comentários.
>     *   Hashtags relevantes e criativos, incluindo alguns que reforçam o tom surreal.
> 4.  **Foco no "Tempo de Viagem":** Mantive o foco nos tempos de viagem fictícios, que são o elemento mais "surreal" e divertido da proposta original.
> 5.  **Características Resumidas e Evocativas:** As descrições dos corpos celestes foram encurtadas e receberam um toque mais descritivo e imaginativo ("centro borbulhante de energia", "brilho que ofusca", "passarela da moda cósmica").
> 6.  **Sugestão Visual:** Adicionei uma sugestão de imagem/carrossel para complementar o texto, pois o Instagram é uma plataforma visual.
> 
> Espero que essa versão te inspire e faça sucesso com seus seguidores! 🌌🚀
